In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import os
import logging
import warnings
import yaml

from replay_benchmarks.utils.conf import load_config, seed_everything
from replay_benchmarks import TrainRunner, InferRunner

import logging
import os
from abc import ABC, abstractmethod
from typing import Tuple

import pandas as pd
from rs_datasets import MovieLens, Netflix

from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.preprocessing.filters import MinCountFilter, NumInteractionsFilter
from replay.splitters import TimeSplitter
from replay.utils import DataFrameLike
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo,
)

from torch.utils.data import DataLoader
from replay.models.nn.sequential.sasrec import (
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionDataset,
)
from replay.models.nn.sequential.bert4rec import (
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionDataset,
)



import logging
import os

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity, schedule

from replay_benchmarks.base_runner import BaseRunner
from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.models.nn.sequential import SasRec, Bert4Rec
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import ValidationMetricsCallback
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.models.nn.sequential.sasrec import (
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionDataset,
)
from replay.models.nn.sequential.bert4rec import (
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionDataset,
)

In [3]:
config_dir = "./replay_benchmarks/configs"
base_config_path = os.path.join(config_dir, "config.yaml")
config = load_config(base_config_path, config_dir)

seed_everything(config["env"]["SEED"])

In [4]:
config = config
model_name = config["model"]["name"]
dataset_name = config["dataset"]["name"]
dataset_cfg = config["dataset"]
model_cfg = config["model"]["params"]
mode = config["mode"]["name"]
item_column = dataset_cfg["feature_schema"]["item_column"]
user_column = dataset_cfg["feature_schema"]["query_column"]
timestamp_column = dataset_cfg["feature_schema"]["timestamp_column"]
tokenizer = None
interactions = None
user_features = None
item_features = None


os.environ["CUDA_DEVICE_ORDER"] = config["env"]["CUDA_DEVICE_ORDER"]
os.environ["OMP_NUM_THREADS"] = config["env"]["OMP_NUM_THREADS"]
os.environ["CUDA_VISIBLE_DEVICES"] = config["env"]["CUDA_VISIBLE_DEVICES"]
os.environ["KAGGLE_USERNAME"] = "recsysaccelerate"
os.environ["KAGGLE_KEY"] = "6363e91b656fea576c39e4f55dcc1d00"

In [5]:
embedding_dim = model_cfg["training_params"]["embedding_dim"]
item_feature_name = "item_id_seq"

tensor_schema =  TensorSchema(
    TensorFeatureInfo(
        name=item_feature_name,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[
            TensorFeatureSource(
                FeatureSource.INTERACTIONS,
                item_column,
            )
        ],
        feature_hint=FeatureHint.ITEM_ID,
        embedding_dim=embedding_dim,
    )
)

tensor_schema

In [6]:
DATASET_MAPPINGS = {
    "zvuk": {"kaggle": "alexxl/zvuk-dataset", "file": "zvuk-interactions.parquet"},
    "megamarket": {"kaggle": "alexxl/megamarket", "file": "megamarket.parquet"},
}
SUPPORTED_RS_DATASETS = ["movielens", "netflix"]

def _download_dataset(
    data_path: str, dataset_name: str, interactions_file: str
):
    """Download dataset from Kaggle or rs_datasets."""
    if dataset_name in DATASET_MAPPINGS:
        _download_kaggle_dataset(data_path, dataset_name, interactions_file)
    elif any(ds in dataset_name for ds in SUPPORTED_RS_DATASETS):
        _download_rs_dataset(data_path, dataset_name, interactions_file)
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")

def _download_kaggle_dataset(
    data_path: str, dataset_name: str, interactions_file: str
) -> None:
    from kaggle.api.kaggle_api_extended import KaggleApi

    """Download dataset from Kaggle."""
    kaggle_info = DATASET_MAPPINGS[dataset_name]
    kaggle_dataset = kaggle_info["kaggle"]
    raw_data_file = os.path.join(data_path, kaggle_info["file"])

    os.environ.setdefault("KAGGLE_USERNAME", "recsysaccelerate")
    os.environ.setdefault("KAGGLE_KEY", "6363e91b656fea576c39e4f55dcc1d00")

    api = KaggleApi()
    api.authenticate()

    api.dataset_download_files(kaggle_dataset, path=data_path, unzip=True)
    logging.info(f"Dataset downloaded and extracted to {data_path}")

    interactions = pd.read_parquet(raw_data_file)
    interactions[timestamp_column] = interactions[
        timestamp_column
    ].astype("int64")
    if dataset_name == "megamarket":
        interactions = interactions[interactions.event == 2] # take only purchase
    interactions.to_parquet(interactions_file)

def _download_rs_dataset(
    data_path: str, dataset_name: str, interactions_file: str
) -> None:
    """Download dataset from rs_datasets."""
    if "movielens" in dataset_name:
        version = dataset_name.split("_")[1]
        movielens = MovieLens(version=version, path=data_path)
        interactions = movielens.ratings
        interactions = interactions[interactions[dataset_cfg["feature_schema"]["rating_column"]] > dataset_cfg["preprocess"]["min_rating"]]
    elif dataset_name == "netflix":
        netflix = Netflix(path=data_path)
        interactions = pd.concat([netflix.train, netflix.test]).fillna(5).reset_index(drop=True)
        interactions = interactions[interactions[dataset_cfg["feature_schema"]["rating_column"]] > dataset_cfg["preprocess"]["min_rating"]]
        interactions = interactions.sort_values(by=[user_column, timestamp_column])
        interactions[timestamp_column] += interactions.groupby([user_column, timestamp_column]).cumcount()
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")

    interactions[timestamp_column] = interactions[
        timestamp_column
    ].astype("int64")
    interactions.to_parquet(interactions_file)


In [7]:
dataset_name = dataset_cfg["name"]
data_path = dataset_cfg["path"]
interactions_file = os.path.join(data_path, "interactions.parquet")

if not os.path.exists(interactions_file):
    _download_dataset(data_path, dataset_name, interactions_file)

interactions = pd.read_parquet(interactions_file)
interactions.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
print(interactions.describe())

            user_id       item_id        rating     timestamp
count  1.000209e+06  1.000209e+06  1.000209e+06  1.000209e+06
mean   3.024512e+03  1.865540e+03  3.581564e+00  9.722437e+08
std    1.728413e+03  1.096041e+03  1.117102e+00  1.215256e+07
min    1.000000e+00  1.000000e+00  1.000000e+00  9.567039e+08
25%    1.506000e+03  1.030000e+03  3.000000e+00  9.653026e+08
50%    3.070000e+03  1.835000e+03  4.000000e+00  9.730180e+08
75%    4.476000e+03  2.770000e+03  4.000000e+00  9.752209e+08
max    6.040000e+03  3.952000e+03  5.000000e+00  1.046455e+09


In [9]:
print(interactions.groupby(user_column).size().min(), interactions.groupby(item_column).size().min())

interactions = MinCountFilter(
    num_entries=dataset_cfg["preprocess"]["min_users_per_item"],
    groupby_column=item_column,
).transform(interactions)

interactions = MinCountFilter(
    num_entries=dataset_cfg["preprocess"]["min_items_per_user"],
    groupby_column=user_column,
).transform(interactions)


interactions.groupby(user_column).size().min(), interactions.groupby(item_column).size().min()

20 1


(18, 5)

In [10]:
interactions.describe()

,user_id,item_id,rating,timestamp
count,999611.000000,999611.000000,999611.000000,9.996110e+05
mean,3024.576537,1865.439428,3.581974,9.722409e+08
std,1728.436705,1095.976336,1.116894,1.214827e+07
min,1.000000,1.000000,1.000000,9.567039e+08
25%,1506.000000,1030.000000,3.000000,9.653025e+08
50%,3070.000000,1835.000000,4.000000,9.730170e+08
75%,4477.000000,2770.000000,4.000000,9.752208e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


In [11]:
splitter = TimeSplitter(
    time_threshold=dataset_cfg["preprocess"]["global_split_ratio"],
    drop_cold_users=True,
    drop_cold_items=True,
    item_column=item_column,
    query_column=user_column,
    timestamp_column=timestamp_column,
)

# train_events, validation_events, validation_gt, test_events, test_gt = (
#     _split_data(splitter, interactions)
# )

test_events, test_gt = splitter.split(interactions)
validation_events, validation_gt = splitter.split(test_events)
train_events = validation_events

test_gt = test_gt[test_gt[item_column].isin(train_events[item_column])]
test_gt = test_gt[test_gt[user_column].isin(train_events[user_column])]


# Limit number of gt events in val and test only if max_num_test_interactions is not null
max_test_interactions = dataset_cfg["preprocess"]["max_num_test_interactions"]
print(f"Distribution of seq_len in validation:\n{validation_gt.groupby(user_column)[item_column].agg('count').describe()}.")
print(f"Distribution of seq_len in test:\n{test_gt.groupby(user_column)[item_column].agg('count').describe()}.")
if max_test_interactions is not None:
    
    validation_gt = NumInteractionsFilter(
        num_interactions=max_test_interactions,
        first=True,
        query_column=user_column,
        item_column=item_column,
        timestamp_column=timestamp_column,
    ).transform(validation_gt)
    print(f"Distribution of seq_len in validation  after filtering:\n{validation_gt.groupby(user_column)[item_column].agg('count').describe()}.")

    test_gt = NumInteractionsFilter(
        num_interactions=max_test_interactions,
        first=True,
        query_column=user_column,
        item_column=item_column,
        timestamp_column=timestamp_column,
    ).transform(test_gt)


Distribution of seq_len in validation:
count     489.000000
mean       40.132924
std        84.792205
min         1.000000
25%         3.000000
50%         8.000000
75%        36.000000
max      1002.000000
Name: item_id, dtype: float64.
Distribution of seq_len in test:
count    1022.000000
mean       79.366928
std       111.068752
min         1.000000
25%        13.000000
50%        37.000000
75%       101.000000
max      1010.000000
Name: item_id, dtype: float64.


In [12]:
test_gt.head()

,user_id,item_id,rating,timestamp
153950,990,2706,3,978137357
153914,990,2826,3,978137357
153908,990,2492,3,978137357
153923,990,1127,4,978137357
153915,990,2683,4,978137392


In [13]:
train_events['timestamp'].max() <= validation_events['timestamp'].min()

False

In [14]:
timestamp_column, item_column, user_column

('timestamp', 'item_id', 'user_id')

In [15]:
def test_splitting(events, gt, name=''):
    if events[timestamp_column].max() > gt[timestamp_column].min():
        print("Problem with time points in", name)
    if len(set(gt[user_column].unique().tolist()) - set(events[user_column].unique().tolist())) > 0:
        print("Problem with cold users in", name)
    if len(set(gt[item_column].unique().tolist()) - set(events[item_column].unique().tolist())) > 0:
        print("Problem with cold items in", name)


test_splitting(train_events, test_gt, "train events, test gt")
test_splitting(train_events, validation_gt, "train events, valid gt")

In [16]:
set([1, 2, 3])  - set({1, 2, 3, 4})

set()

In [17]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    """Prepare the feature schema based on whether ground truth is needed."""
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column=user_column,
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column=item_column,
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    return base_features + FeatureSchema(
        [
            FeatureInfo(
                column=timestamp_column,
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )

feature_schema = prepare_feature_schema(is_ground_truth=False)
ground_truth_schema = prepare_feature_schema(is_ground_truth=True)

train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt_dataset = Dataset(
    feature_schema=ground_truth_schema,
    interactions=validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)
test_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt_dataset = Dataset(
    feature_schema=ground_truth_schema,
    interactions=test_gt,
    check_consistency=True,
    categorical_encoded=False,
)


In [18]:
tokenizer = SequenceTokenizer(
    tensor_schema, allow_collect_to_master=True, handle_unknown_rule="drop"
)
tokenizer.fit(train_dataset)

seq_train_dataset = tokenizer.transform(train_dataset)
# seq_validation_dataset, seq_validation_gt = _prepare_sequential_validation(
#     validation_dataset, validation_gt
# )

seq_validation_dataset = tokenizer.transform(validation_dataset)
seq_validation_gt = tokenizer.transform(
    validation_gt_dataset, [tensor_schema.item_id_feature_name]
)

seq_validation_dataset, seq_validation_gt = SequentialDataset.keep_common_query_ids(
            seq_validation_dataset, seq_validation_gt
)


test_query_ids = test_gt_dataset.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)[
    user_column
].values
seq_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(
    test_query_ids_np
)
# seq_test_dataset = self._prepare_sequential_test(test_dataset, test_gt)


In [19]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

print(len(tokenizer.query_id_encoder.mapping["user_id"]), len(tokenizer.item_id_encoder.mapping['item_id']))

{'user_id': {6040: 0, 6039: 1, 6038: 2, 6037: 3, 6036: 4, 6035: 5, 6034: 6, 6033: 7, 6032: 8, 6031: 9, 6030: 10, 6029: 11, 6028: 12, 6027: 13, 6026: 14, 6025: 15, 6024: 16, 6023: 17, 6022: 18, 6021: 19, 6020: 20, 6019: 21, 6018: 22, 6017: 23, 6016: 24, 6015: 25, 6014: 26, 6013: 27, 6012: 28, 6011: 29, 6010: 30, 6009: 31, 6007: 32, 6008: 33, 6006: 34, 6005: 35, 6004: 36, 6003: 37, 6002: 38, 6001: 39, 6000: 40, 5999: 41, 5998: 42, 5997: 43, 5996: 44, 5995: 45, 5994: 46, 5993: 47, 5992: 48, 5991: 49, 5990: 50, 5989: 51, 5988: 52, 5987: 53, 5986: 54, 5984: 55, 5983: 56, 5982: 57, 5981: 58, 5979: 59, 5980: 60, 5978: 61, 5977: 62, 5976: 63, 5975: 64, 5974: 65, 5973: 66, 5972: 67, 5971: 68, 5970: 69, 5969: 70, 5968: 71, 5967: 72, 5966: 73, 5965: 74, 5964: 75, 5963: 76, 5962: 77, 5961: 78, 5960: 79, 5959: 80, 5958: 81, 5957: 82, 5956: 83, 5955: 84, 5954: 85, 5953: 86, 5952: 87, 5951: 88, 5950: 89, 5948: 90, 5947: 91, 5946: 92, 5945: 93, 5944: 94, 5943: 95, 5942: 96, 5941: 97, 5940: 98, 5939: 9

In [20]:
dataset_mapping = {
    "sasrec": (
        SasRecTrainingDataset,
        SasRecValidationDataset,
        SasRecPredictionDataset,
    ),
    "bert4rec": (
        Bert4RecTrainingDataset,
        Bert4RecValidationDataset,
        Bert4RecPredictionDataset,
    ),
}

if model_name.lower() in dataset_mapping:
    TrainingDataset, ValidationDataset, PredictionDataset = dataset_mapping[
        model_name.lower()
    ]
else:
    raise ValueError(
        f"Unsupported model type for dataloaders: {model_name}"
    )

common_params = {
    "batch_size": model_cfg["training_params"]["batch_size"],
    "num_workers": model_cfg["training_params"]["num_workers"],
    "pin_memory": True,
}

train_dataloader = DataLoader(
    dataset=TrainingDataset(
        seq_train_dataset,
        max_sequence_length=model_cfg["model_params"]["max_seq_len"],
    ),
    shuffle=True,
    **common_params,
)
val_dataloader = DataLoader(
    dataset=ValidationDataset(
        seq_validation_dataset,
        seq_validation_gt,
        seq_train_dataset,
        max_sequence_length=model_cfg["model_params"]["max_seq_len"],
    ),
    **common_params,
)
prediction_dataloader = DataLoader(
    dataset=PredictionDataset(
        seq_test_dataset,
        max_sequence_length=model_cfg["model_params"]["max_seq_len"],
    ),
    **common_params,
)


In [21]:
model_config = {
    "tensor_schema": tensor_schema,
    "optimizer_factory": FatOptimizerFactory(
        learning_rate=model_cfg["training_params"]["learning_rate"]
    ),
}
model_config.update(model_cfg["model_params"])

if model_name.lower() == "sasrec":
    model = SasRec(**model_config)
elif model_name.lower() == "bert4rec":
    model = Bert4Rec(**model_config)
else:
    raise ValueError(f"Unsupported model type: {model_name}")

In [22]:
model._model.num_blocks, model._model.num_heads, model._model.hidden_size, model._model.max_len, model._model.dropout,

(2, 2, 300, 200, 0.5)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath=config["paths"]["checkpoint_dir"],
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

# validation_metrics_callback = ValidationMetricsCallback(
#     metrics=config["metrics"]["types"],
#     ks=config["metrics"]["ks"],
#     item_count=train_dataset.item_count,
#     postprocessors=[RemoveSeenItems(seq_validation_dataset)],
# )

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(seq_validation_dataset)]
)

trainer = L.Trainer(
    max_epochs=model_cfg["training_params"]["max_epochs"],
    callbacks=[checkpoint_callback, validation_metrics_callback],
)

trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.se

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

k              1        10        20         5
map     0.010225  0.004799  0.004213  0.006178
ndcg    0.010225  0.012489  0.014852  0.011929
recall  0.001169  0.004432  0.009039  0.002671



/usr/local/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Epoch 2, global step 33: 'recall@10' was not in top 1


k              1        10        20         5
map     0.145194  0.070679  0.061700  0.090295
ndcg    0.145194  0.122818  0.127504  0.129151
recall  0.006152  0.034238  0.069411  0.022235



Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 3, global step 44: 'recall@10' reached 0.04017 (best 0.04017), saving model to '/root/RePlay-Accelerated/replay_benchmarks/artifacts/checkpoints/epoch=3-step=44-v1.ckpt' as top 1


k              1        10        20         5
map     0.120654  0.069530  0.059356  0.081084
ndcg    0.120654  0.123780  0.125654  0.120974
recall  0.004143  0.040173  0.077692  0.017718

